In [1]:
import tensorflow as tf
import keras

In [2]:
# Functional model defined(1 input and 1 output)
inputs = keras.Input(shape=(3,),name="my_input") # Input layer defined in "inputs" object
features = keras.layers.Dense(units=64, activation="relu")(inputs) # Feature layer whose input is the input layer defined earlier (hidden layer defined in "features" object)
outputs = keras.layers.Dense(units=1,activation="softmax")(features) # Output layer whose input is the feature layer defined earlier (output layer defined in "outputs" object)
model = keras.Model(inputs=inputs,outputs=outputs)

In [3]:
inputs.shape # The batch size of the input layer is variable(can have any value, since its presented as None)
# inputs object is called a symbolic tensor - doesn't contain actual data but it contains the info about the specification of the actual tensors that the model will use as inputs

TensorShape([None, 3])

In [4]:
features.shape 

TensorShape([None, 64])

In [5]:
outputs.shape

TensorShape([None, 1])

In [6]:
from keras import Input, Model
from keras.layers import Dense,Concatenate

In [7]:
# Model for problem statement: 3 inputs --> Title of the ticket, text body of the ticket and tags added by the user
#2 outputs --> Priority score of the ticket, department which will handle the ticket

# Vocabulary is used to convert the text inputs into arrays

vocabulary_size =10000
num_tags = 100
department_num = 4

title_input = Input(shape=(vocabulary_size,),name="title input")
body_input = Input(shape=(vocabulary_size,),name="body input")
tags_input = Input(shape=(num_tags,),name="tags input")

features_concatenate = Concatenate()([title_input, body_input,tags_input]) # Layer to concatenate all the 3 inputs together
features_layer = Dense(units=64,activation="relu")(features_concatenate) # Hidden layer to use the inputs for prediction
priority_outputs = Dense(units=1,activation="softmax",name="priority_outputs")(features_layer) # Output layer to predict the priority score
department_outputs = Dense(units=department_num,activation="softmax",name="department_outputs")(features_layer) # Output layer to predict the department(units = number of departments we have , since its multiclass classification) 

model1 = Model(inputs=[title_input,body_input,tags_input],outputs=[priority_outputs,department_outputs])

In [8]:
model1.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 title input (InputLayer)       [(None, 10000)]      0           []                               
                                                                                                  
 body input (InputLayer)        [(None, 10000)]      0           []                               
                                                                                                  
 tags input (InputLayer)        [(None, 100)]        0           []                               
                                                                                                  
 concatenate (Concatenate)      (None, 20100)        0           ['title input[0][0]',            
                                                                  'body input[0][0]',       

In [9]:
# Creating random inputs for our model to train
import numpy as np
num_samples = 1500

title_data = np.random.randint(0,2,size=(num_samples,vocabulary_size)) # lowest value , highest value , size
body_data = np.random.randint(0,2,size=(num_samples,vocabulary_size))
tags_data = np.random.randint(0,2,size=(num_samples,num_tags))

priority_data = np.random.random(size=(num_samples,1))
department_data = np.random.randint(0,2,size=(num_samples,department_num))

In [10]:
# Model compilation
model1.compile(loss=["mean_squared_error","categorical_crossentropy"],optimizer="rmsprop",metrics=["mae","accuracy"]) # 2 losses and 2 metrics for 2 outputs

In [11]:
history = model1.fit([title_data,body_data,tags_data],[priority_data,department_data],epochs=20)

# If we don't have the order of the inputs and outputs, we can pass as a dictionary

#model1.fit({"title_input":title_data, "body_input" : body_data, "tags_input":tags_data} , {"priority_outputs":priority_data, "department_outputs":department_data},epochs=20)
history

Epoch 1/20
47/47 [==============================] - 2s 13ms/step - loss: 35.3374 - priority_outputs_loss: 0.3204 - department_outputs_loss: 35.0170 - priority_outputs_mae: 0.4903 - priority_outputs_accuracy: 0.0000e+00 - department_outputs_mae: 0.5051 - department_outputs_accuracy: 0.2660
Epoch 2/20
47/47 [==============================] - 1s 13ms/step - loss: 33.2169 - priority_outputs_loss: 0.3204 - department_outputs_loss: 32.8965 - priority_outputs_mae: 0.4903 - priority_outputs_accuracy: 0.0000e+00 - department_outputs_mae: 0.4947 - department_outputs_accuracy: 0.2333
Epoch 3/20
47/47 [==============================] - 1s 12ms/step - loss: 39.4649 - priority_outputs_loss: 0.3204 - department_outputs_loss: 39.1445 - priority_outputs_mae: 0.4903 - priority_outputs_accuracy: 0.0000e+00 - department_outputs_mae: 0.4887 - department_outputs_accuracy: 0.2573
Epoch 4/20
47/47 [==============================] - 1s 13ms/step - loss: 48.6458 - priority_outputs_loss: 0.3204 - department_outp

In [12]:
history_dict = history.history
history_dict.keys()

dict_keys(['loss', 'priority_outputs_loss', 'department_outputs_loss', 'priority_outputs_mae', 'priority_outputs_accuracy', 'department_outputs_mae', 'department_outputs_accuracy'])

In [24]:
model1.layers

In [27]:
model1.layers[4] # Used to extract the layers from the model and if we want to add an additional output, instead of creating the model from scratch, we can use the exisiting model layers and 
# modeify them or add new layers, to create the new model

In [28]:
model1.layers[4].input # To get the input info about the layer

<KerasTensor: shape=(None, 20100) dtype=float32 (created by layer 'concatenate')>

In [34]:
model1.layers[4].output # To get the output info about the layer --> This is the output of the hidden dense layer

<KerasTensor: shape=(None, 64) dtype=float32 (created by layer 'dense_2')>

In [35]:
# Now we are adding a new output called difficulty_output with 3 classes --> quick,medium and difficult

In [38]:
features_layer_output = model1.layers[4].output
difficulty_output = Dense(units=3,activation="softmax",name="difficulty_output")(features_layer_output)

new_model = Model(inputs=[title_input,body_input,tags_input],outputs=[priority_outputs,department_outputs,difficulty_output])

new_model.compile(optimizer="rmsprop",loss=["mean_squared_error","categorical_crossentropy","categorical_crossentropy"],metrics=["mae","accuracy"])

# Random input data created for difficulty
difficulty_num = 3
difficulty_data = np.random.randint(0,2,size=(num_samples,difficulty_num))

history_new = new_model.fit({"title input":title_data, "body input" : body_data, "tags input":tags_data} , {"priority_outputs":priority_data, "department_outputs":department_data,"difficulty_output":difficulty_data},epochs=20)
history_new_dict = history_new.history

Epoch 1/20
47/47 [==============================] - 2s 15ms/step - loss: 1162.3920 - priority_outputs_loss: 0.3204 - department_outputs_loss: 319.1971 - difficulty_output_loss: 842.8744 - priority_outputs_mae: 0.4903 - priority_outputs_accuracy: 0.0000e+00 - department_outputs_mae: 0.4950 - department_outputs_accuracy: 0.2533 - difficulty_output_mae: 0.4933 - difficulty_output_accuracy: 0.2860
Epoch 2/20
47/47 [==============================] - 1s 15ms/step - loss: 463.9489 - priority_outputs_loss: 0.3204 - department_outputs_loss: 278.0786 - difficulty_output_loss: 185.5501 - priority_outputs_mae: 0.4903 - priority_outputs_accuracy: 0.0000e+00 - department_outputs_mae: 0.4925 - department_outputs_accuracy: 0.2647 - difficulty_output_mae: 0.4940 - difficulty_output_accuracy: 0.3433
Epoch 3/20
47/47 [==============================] - 1s 15ms/step - loss: 467.6160 - priority_outputs_loss: 0.3204 - department_outputs_loss: 278.1009 - difficulty_output_loss: 189.1947 - priority_outputs_mae

In [39]:
history_new_dict.keys()

dict_keys(['loss', 'priority_outputs_loss', 'department_outputs_loss', 'difficulty_output_loss', 'priority_outputs_mae', 'priority_outputs_accuracy', 'department_outputs_mae', 'department_outputs_accuracy', 'difficulty_output_mae', 'difficulty_output_accuracy'])